# Assignment: Supervised Learning Flow
**Student:** Itzhak D  
**ID (last 4 digits):** 6233

# תקציר המחברת

במחברת זו אנו מנתחים נתוני מחירי בתים במטרה לחזות את ערך הבית הממוצע.
התהליך כולל את השלבים הבאים:
1. **טעינת נתונים** – העלאת קבצי אימון ובדיקה וקריאתם לטבלאות ניתוח.
2. **ניתוח ראשוני** – חקירת התפלגות מחירי הבתים, השוואת משתנים מרכזיים כמו הכנסה מדדית ומיקום גיאוגרפי מול מחיר הבית, ובחינת הקשרים בין כל המשתנים.
3. **חיפוש פרמטרים מיטביים עם ולידציה צולבת** – בחינת השפעת עיבוד נתונים ופרמטרים שונים על ביצועי המודל.
4. **אימון המודל הסופי** – אימון המודל הטוב ביותר שנמצא על כל נתוני האימון.
5. **חיזוי והערכת ביצועים על סט הבדיקה** – חיזוי מחירי בתים בסט הבדיקה ובדיקת ביצועים בעזרת מדדים סטטיסטיים מתאימים.
מטרת המחברת היא להציג תהליך למידה מונחה מובנה, משלב ניתוח הנתונים ועד בחירת המודל הטוב ביותר והערכתו.

In [ ]:
# ===========================
# PART 1 – Setup + File Upload
# ===========================
from google.colab import files
import os
import pandas as pd

# העלאת קבצי train ו-test
uploaded = files.upload()

# שינוי שמות הקבצים למבנה תקני
for fn in uploaded.keys():
    new_name = fn.replace(' ', '_').replace('(', '').replace(')', '')
    os.rename(fn, new_name)
    print(f'File uploaded and renamed: {fn} -> {new_name}')

# טעינת הנתונים
train_file = [f for f in os.listdir() if 'train' in f.lower()][0]
test_file = [f for f in os.listdir() if 'test' in f.lower()][0]
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)
display(train_df.head())
display(test_df.head())

In [ ]:
# ===========================
# PART 2 – Exploratory Data Analysis (EDA)
# ===========================
import matplotlib.pyplot as plt
import seaborn as sns

# התפלגות מחירי בתים
plt.figure(figsize=(6,4))
plt.hist(train_df['MedHouseVal'].dropna(), bins=30)
plt.title('House Price Distribution')
plt.xlabel('Median House Value')
plt.ylabel('Number of Samples')
plt.tight_layout()
plt.show()

# השוואת הכנסה מדדית מול מחיר הבית
plt.figure(figsize=(6,4))
plt.scatter(train_df['MedInc'], train_df['MedHouseVal'], s=8, alpha=0.6)
plt.title('Median Income vs House Price')
plt.xlabel('Median Income')
plt.ylabel('Median House Value')
plt.tight_layout()
plt.show()

# קו רוחב מול מחיר הבית
plt.figure(figsize=(6,4))
plt.scatter(train_df['Latitude'], train_df['MedHouseVal'], s=8, alpha=0.6)
plt.title('Latitude vs House Price')
plt.xlabel('Latitude')
plt.ylabel('Median House Value')
plt.tight_layout()
plt.show()

# מפת חום של קורלציות
plt.figure(figsize=(10,8))
sns.heatmap(train_df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# ===========================
# PART 3 – Grid Search + 5-Fold Cross Validation
# ===========================
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

# הפרדת מאפיינים ויעד
X_train = train_df.drop('MedHouseVal', axis=1)
y_train = train_df['MedHouseVal']

# בניית pipeline
pipe = Pipeline([('scaler', 'passthrough'), ('model', Ridge())])

# טווחי פרמטרים
param_grid = {
    'scaler': ['passthrough', StandardScaler()],
    'model__alpha': [1.0, 10.0],
    'model__fit_intercept': [True, False]
}

# קיפול נתונים ל-5 חלקים
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(pipe, param_grid=param_grid, scoring='r2', cv=cv, n_jobs=-1, return_train_score=True)
grid.fit(X_train, y_train)

# הצגת התוצאות
cv_results = pd.DataFrame(grid.cv_results_)
cols = ['param_scaler','param_model__alpha','param_model__fit_intercept', 'mean_test_score','std_test_score','mean_train_score','rank_test_score']
display(cv_results[cols].sort_values('mean_test_score', ascending=False).head())

In [ ]:
# ===========================
# PART 4 – Train Final Model
# ===========================
best_est = grid.best_estimator_
print('Best Estimator:', best_est)
best_est.fit(X_train, y_train)

In [ ]:
# ===========================
# PART 5 – Predict on Test & Evaluate
# ===========================
from sklearn.metrics import r2_score

if 'MedHouseVal' in test_df.columns:
    X_test = test_df.drop('MedHouseVal', axis=1)
    y_test = test_df['MedHouseVal']
    test_pred = best_est.predict(X_test)
    test_r2 = r2_score(y_test, test_pred)
    print('R^2 on test set:', round(test_r2, 4))
else:
    X_test = test_df.copy()
    test_pred = best_est.predict(X_test)
    print('Test set has no target values; only predictions are shown')

# הצגת חמשת התחזיות הראשונות
pred_samples = pd.DataFrame({**{c: X_test[c].iloc[:5].values for c in X_test.columns}, 'y_pred': test_pred[:5]})
if 'MedHouseVal' in test_df.columns:
    pred_samples['y_true'] = y_test.iloc[:5].values
display(pred_samples)